In [127]:
import numpy as np
import pandas as pd
import os

### Read the file

In [128]:
directory_path = os.getcwd()
folder_name = os.path.basename(directory_path)
directory_path = os.getcwd()
raw_df = pd.read_feather(os.path.join(directory_path, 'chars60_raw_imputed.feather'))

In [129]:
raw_df.columns

Index(['gvkey', 'permno', 'sic', 'ret', 'exchcd', 'shrcd', 'adm', 'bm_ia',
       'herf', 'hire', 'me_ia', 'baspread', 'beta', 'ill', 'maxret', 'mom12m',
       'mom1m', 'mom36m', 'mom60m', 'mom6m', 're', 'rvar_capm', 'rvar_ff3',
       'rvar_mean', 'seas1a', 'std_dolvol', 'std_turn', 'zerotrade', 'me',
       'dy', 'turn', 'dolvol', 'abr', 'sue', 'cinvest', 'nincr', 'pscore',
       'acc', 'bm', 'agr', 'alm', 'ato', 'cash', 'cashdebt', 'cfp', 'chcsho',
       'chpm', 'chtx', 'depr', 'ep', 'gma', 'grltnoa', 'lev', 'lgr', 'ni',
       'noa', 'op', 'pctacc', 'pm', 'rd_sale', 'rdm', 'rna', 'roa', 'roe',
       'rsup', 'sgr', 'sp', 'date', 'ffi49'],
      dtype='object')

In [130]:
#Check missing value
raw_tf = raw_df.isnull().any()
raw_tf.where(raw_tf == True).dropna().index.to_list()
# There're too much missing value before 1972, so we focus on the data after 1972.
raw_df['year'] = raw_df['date'].dt.year
raw_df = raw_df[raw_df['year'] >= 1972]
raw_df = raw_df.drop(['year'], axis=1)
raw_tf = raw_df.isnull().any()
raw_tf.where(raw_tf == True).dropna().index.to_list()

[]

In [131]:
raw_df

,gvkey,permno,sic,ret,exchcd,shrcd,adm,bm_ia,herf,hire,...,rd_sale,rdm,rna,roa,roe,rsup,sgr,sp,date,ffi49
136,1010,10006,3743,0.004854,1.0,10.0,0.000006,0.000046,0.718884,-0.049123,...,0.007357,0.000009,0.106823,-0.000011,0.096217,0.000047,0.042217,0.001158,1972-01-31,25
137,1010,10006,3743,-0.007729,1.0,10.0,0.000006,0.000042,0.718884,-0.049123,...,0.007357,0.000008,0.106823,-0.000011,0.096217,0.000047,0.042217,0.001152,1972-02-29,25
138,1010,10006,3743,-0.068966,1.0,10.0,0.000006,0.000059,0.718884,-0.049123,...,0.007357,0.000009,0.106823,-0.000011,0.096217,0.000048,0.042217,0.001175,1972-03-31,25
139,1010,10006,3743,-0.068783,1.0,10.0,0.000007,0.000122,0.718884,-0.049123,...,0.007357,0.000009,0.106823,-0.000011,0.096217,0.000051,0.042217,0.001262,1972-04-30,25
140,1010,10006,3743,0.025000,1.0,10.0,0.000006,0.000164,0.718079,-0.099815,...,0.006949,0.000009,0.090818,0.000007,0.080707,-0.000004,-0.003038,0.001351,1972-05-31,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
551140,260778,90272,6324,-0.057476,1.0,11.0,0.000005,-0.000689,0.092306,0.348315,...,0.000000,0.000000,0.053990,0.015538,0.043113,0.000146,0.272945,0.001559,2019-08-31,46
551141,260778,90272,6324,-0.042735,1.0,11.0,0.000006,-0.000671,0.092306,0.348315,...,0.000000,0.000000,0.053990,0.015538,0.043113,0.000155,0.272945,0.001654,2019-09-30,46
551142,260778,90272,6324,0.144423,1.0,11.0,0.000006,-0.000657,0.092306,0.348315,...,0.000000,0.000000,0.053990,0.015538,0.043113,0.000162,0.272945,0.001728,2019-10-31,46
551143,260778,90272,6324,0.085873,1.0,11.0,0.000006,-0.000699,0.092306,0.348315,...,0.000000,0.000000,0.067523,0.018249,0.054882,0.000159,0.380853,0.001668,2019-11-30,46


In [132]:
# add macro file
macro_df = pd.read_csv(os.path.join(directory_path, 'Macro.csv'))
macro_df = macro_df[macro_df['yyyymm']>=197201]

In [133]:
# merge them together
raw_df['yyyymm'] = raw_df['date'].dt.strftime("%Y%m").apply(int)
total_df = pd.merge(raw_df,macro_df,how='inner',on='yyyymm')
total_df.drop(columns='yyyymm',inplace=True)

In [134]:
total_df

,gvkey,permno,sic,ret,exchcd,shrcd,adm,bm_ia,herf,hire,...,date,ffi49,b/m,tbl,ntis,svar,d/p,e/p,dfy,tms
0,1010,10006,3743,0.004854,1.0,10.0,0.000006,0.000046,0.718884,-0.049123,...,1972-01-31,25,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
1,1040,15763,3949,0.111399,1.0,10.0,0.000037,-0.000224,0.163326,0.033872,...,1972-01-31,6,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
2,1043,18980,5040,0.080000,1.0,11.0,0.000037,0.000088,1.000000,-0.092000,...,1972-01-31,42,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
3,1045,21020,4512,0.042945,1.0,11.0,0.000031,-0.000685,0.062520,-0.033342,...,1972-01-31,41,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
4,1070,21549,3714,0.191489,2.0,10.0,0.000037,0.001247,0.248527,0.000000,...,1972-01-31,23,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462751,188255,12981,2430,0.048165,1.0,11.0,0.000025,0.000052,0.262476,0.000000,...,2013-12-31,17,0.304408,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371
462752,189491,13035,3561,0.001157,1.0,11.0,0.000003,-0.000044,0.157914,0.016000,...,2013-12-31,21,0.304408,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371
462753,199356,13168,7370,-0.062160,3.0,11.0,0.000015,-0.000239,0.267858,0.260000,...,2013-12-31,36,0.304408,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371
462754,260774,90199,6500,0.084983,1.0,11.0,0.000005,-0.000063,0.476217,0.088235,...,2013-12-31,47,0.304408,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371


### pre-Process the Data

In [135]:
len(total_df['ffi49'].value_counts())

48

In [136]:
len(total_df['sic'].value_counts())
# 'sic' and 'ffi49' are all industry information, so they might hvae something in common.
#  Here we choose to drop 'sic', since there're too much types.

303

In [137]:
# set date as Index
total_df.set_index('date', inplace=True)

In [138]:
# also, gvkey and permno are codes for each stock. Here we want to focus on the importance of each characteristic.
# So we drop them.
y_total = total_df['ret']
total_df = total_df.drop(columns={'sic','permno','gvkey','ret'})

In [139]:
y_total

date
1972-01-31    0.004854
1972-01-31    0.111399
1972-01-31    0.080000
1972-01-31    0.042945
1972-01-31    0.191489
                ...   
2013-12-31    0.048165
2013-12-31    0.001157
2013-12-31   -0.062160
2013-12-31    0.084983
2013-12-31   -0.052221
Name: ret, Length: 462756, dtype: float64

In [140]:
df

,adm,bm_ia,herf,hire,me_ia,baspread,beta,ill,maxret,mom12m,...,sgr,sp,b/m,tbl,ntis,svar,d/p,e/p,dfy,tms
date,,,,,,,,,,,,,,,,,,,,,
1972-01-31,0.000006,0.000046,0.718884,-0.049123,1.259719e+05,0.009656,0.794048,1.240770,0.049180,0.121682,...,0.042217,0.001158,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
1972-01-31,0.000037,-0.000224,0.163326,0.033872,5.186702e+05,0.015009,1.409503,13.821094,0.041420,0.438430,...,0.053868,0.000733,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
1972-01-31,0.000037,0.000088,1.000000,-0.092000,-4.160822e+04,0.030370,2.205532,12.353322,0.083700,0.253863,...,-0.019391,0.001492,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
1972-01-31,0.000031,-0.000685,0.062520,-0.033342,5.607096e+05,0.022826,1.588084,22.634887,0.049296,0.750638,...,0.096634,0.001145,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
1972-01-31,0.000037,0.001247,0.248527,0.000000,-2.731388e+06,0.042447,0.720158,3.520096,0.116279,0.516129,...,0.141075,0.006796,0.635357,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-31,0.000025,0.000052,0.262476,0.000000,1.976499e+06,0.013114,1.669907,494.065086,0.058240,0.443785,...,0.058325,0.000505,0.304408,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371
2013-12-31,0.000003,-0.000044,0.157914,0.016000,-7.224876e+06,0.011212,1.065726,1062.994422,0.124438,0.344219,...,-0.024524,0.000586,0.304408,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371
2013-12-31,0.000015,-0.000239,0.267858,0.260000,-9.982225e+07,0.015552,1.345975,479.958631,0.058048,1.165881,...,1.255297,0.000075,0.304408,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371


#### transform the dummy variables and normalization

In [141]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
# we use onehotencoder to encode the dummy variables
def onehot(columns):
    ohe = OneHotEncoder()
    trans = ohe.fit_transform(total_df[[columns]])
    
    return trans.toarray()

In [142]:
d_ffi49 = onehot('ffi49')
d_exchcd = onehot('exchcd')
d_shrcd = onehot('shrcd')
total_df = total_df.drop(columns={'ffi49','exchcd','shrcd'})
features = total_df.columns.tolist()

In [220]:
d_ffi49.tolist()

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [143]:
# then we merge them together
total_df['ffi49'] = d_ffi49.tolist()
total_df['exchcd'] = d_exchcd.tolist()
total_df['shrcd'] = d_shrcd.tolist()
total_df

,adm,bm_ia,herf,hire,me_ia,baspread,beta,ill,maxret,mom12m,...,tbl,ntis,svar,d/p,e/p,dfy,tms,ffi49,exchcd,shrcd
date,,,,,,,,,,,,,,,,,,,,,
1972-01-31,0.000006,0.000046,0.718884,-0.049123,1.259719e+05,0.009656,0.794048,1.240770,0.049180,0.121682,...,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[1.0, 0.0]"
1972-01-31,0.000037,-0.000224,0.163326,0.033872,5.186702e+05,0.015009,1.409503,13.821094,0.041420,0.438430,...,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[1.0, 0.0]"
1972-01-31,0.000037,0.000088,1.000000,-0.092000,-4.160822e+04,0.030370,2.205532,12.353322,0.083700,0.253863,...,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 1.0]"
1972-01-31,0.000031,-0.000685,0.062520,-0.033342,5.607096e+05,0.022826,1.588084,22.634887,0.049296,0.750638,...,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 1.0]"
1972-01-31,0.000037,0.001247,0.248527,0.000000,-2.731388e+06,0.042447,0.720158,3.520096,0.116279,0.516129,...,0.0338,0.033075,0.000494,1.529644,1.258123,0.0104,0.0268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0]","[1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-31,0.000025,0.000052,0.262476,0.000000,1.976499e+06,0.013114,1.669907,494.065086,0.058240,0.443785,...,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 1.0]"
2013-12-31,0.000003,-0.000044,0.157914,0.016000,-7.224876e+06,0.011212,1.065726,1062.994422,0.124438,0.344219,...,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0]","[0.0, 1.0]"
2013-12-31,0.000015,-0.000239,0.267858,0.260000,-9.982225e+07,0.015552,1.345975,479.958631,0.058048,1.165881,...,0.0007,0.012144,0.000746,1.722817,1.265919,0.0076,0.0371,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0]","[0.0, 1.0]"


## Split the Data(还没有rolling）and Random Forest
### Total 49 years: 32 years for training, 6 years for validation and 6 years for testing.

In [145]:
from sklearn.preprocessing import StandardScaler

In [234]:
train_x = total_df['1972-01-01':'1988-12-31']
train_y = y_total['1972-01-01':'1988-12-31']
val_x = total_df['1989-01-01':'1995-12-31']
val_y = y_total['1989-01-01':'1995-12-31']
test_y = y_total['1996-01-01':'2001-12-31']
test_x = total_df['1996-01-01':'2001-12-31']

train_ffi49 = train_x['ffi49']
val_ffi49 = val_x['ffi49']
test_ffi49 = test_x['ffi49']
train_exchcd = train_x['exchcd']
val_exchcd = val_x['exchcd']
test_exchcd = test_x['exchcd']
train_shrcd = train_x['shrcd']
val_shrcd = val_x['shrcd']
test_shrcd = test_x['shrcd']

# Standarlize
scaler = StandardScaler()
normtrain_x = scaler.fit_transform(train_x[features])
normval_x = scaler.transform(val_x[features])
normtest_x = scaler.transform(test_x[features])

normtrain_x_df = pd.DataFrame(normtrain_x, index = train_x.index, columns=features)
normval_x_df = pd.DataFrame(normval_x, index = val_x.index, columns=features)
normtest_x_df = pd.DataFrame(normtest_x, index = test_x.index, columns=features)

# get the dataframe
normtrain_x_df['ffi49'] = train_ffi49
normtrain_x_df['exchcd'] = train_exchcd
normtrain_x_df['shrcd'] = train_shrcd

normval_x_df['ffi49'] = val_ffi49
normval_x_df['exchcd'] = val_exchcd
normval_x_df['shrcd'] = val_shrcd

normtest_x_df['ffi49'] = test_ffi49
normtest_x_df['exchcd'] = test_exchcd
normtest_x_df['shrcd'] = test_shrcd

# get the array
normtrain_x = np.concatenate((normtrain_x,np.array(train_ffi49.tolist()),np.array(train_exchcd.tolist()),np.array(train_shrcd.tolist())),axis=1)
normval_x = np.concatenate((normval_x,np.array(val_ffi49.tolist()),np.array(val_exchcd.tolist()),np.array(val_shrcd.tolist())),axis=1)
normtest_x = np.concatenate((normtest_x,np.array(test_ffi49.tolist()),np.array(test_exchcd.tolist()),np.array(test_shrcd.tolist())),axis=1)



/Users/liuxuyang/opt/anaconda3/envs/mytensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  """Entry point for launching an IPython kernel.
/Users/liuxuyang/opt/anaconda3/envs/mytensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  
/Users/liuxuyang/opt/anaconda3/envs/mytensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liuxuyang/opt/anaconda3/envs/mytensorflow/lib/python3.7/site-packages/ip

In [121]:
# Now we start training in random forest.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt

In [235]:
def RFReg(X_train, y_train, X_valid, y_valid):
    rf_best = RandomForestRegressor().fit(X_train, y_train)
    for L in range(1,10): # 调整超参L
        for B in range(1,10): # 调整超参B
            rf_temp = RandomForestRegressor(n_estimators=10, max_depth=None).fit(X_train, y_train)
            if rf_temp.score(X_valid, y_valid)> rf_best.score(X_valid, y_valid): # 保留使得模型得分最好的B\L
                L_best = L
                B_best = B
                rf_best = RandomForestRegressor(n_estimators=B_best, max_depth=L_best).fit(X_train, y_train) # 用最好的参数训练模型
    return rf_best

In [ ]:
rf = RFReg(normtrain_x,train_y,normval_x,val_y)

In [237]:
rf = RandomForestRegressor(n_estimators=10, max_depth=None).fit(normtrain_x, train_y)

In [241]:
def calculate_oos(test_y,prediction_y):
    denominator = (test_y ** 2).sum()
    numerator = ((test_y-prediction_y)**2 ).sum()
    roos = 1 - numerator / denominator
    return roos
    

In [244]:
rf.score(normval_x,val_y)

-0.24805160393271342

In [245]:
rf2 = RandomForestRegressor(n_estimators=10, max_depth=5).fit(normtrain_x, train_y)

In [246]:
rf2.score(normval_x,val_y)

-0.07517625560231656

In [248]:
rf3 = RandomForestRegressor(n_estimators=100, max_depth=2).fit(normtrain_x, train_y)
rf3.score(normval_x,val_y)
# 他这个score是一般的R2，不是out of sample R2

-0.03961656791298496

In [256]:
rf4 = RandomForestRegressor(n_estimators=300, max_depth=5).fit(normtrain_x, train_y)
rf4.score(normval_x,val_y)
rf4.score(normtrain_x, train_y)

-0.07333211104093529

In [257]:
rf4.score(normtrain_x, train_y)

0.1571841163579063

In [260]:
rf5 = RandomForestRegressor(n_estimators=300, max_depth=15).fit(normtrain_x, train_y)
print(rf5.score(normval_x,val_y))
print(rf5.score(normtrain_x, train_y))

-0.14525348170886732
0.48534684554820584


In [261]:
rf5 = RandomForestRegressor(n_estimators=300, max_depth=10).fit(normtrain_x, train_y)
print(rf5.score(normval_x,val_y))
print(rf5.score(normtrain_x, train_y))

-0.10153070286255139
0.3121668745743542


In [262]:
rf5 = RandomForestRegressor(n_estimators=350, max_depth=10).fit(normtrain_x, train_y)
print(rf5.score(normval_x,val_y))
print(rf5.score(normtrain_x, train_y))

-0.10060809434956619
0.31266193627461547


In [ ]:
rf5 = RandomForestRegressor(n_estimators=350, max_depth=5).fit(normtrain_x, train_y)
print(rf5.score(normval_x,val_y))
print(rf5.score(normtrain_x, train_y))

In [259]:
rf5.score(normval_x,val_y)

-0.09913698721093733

In [253]:
pre_y=rf3.predict(normval_x)
r22 = calculate_oos(val_y,pre_y)
r22

-0.014844882524612801

In [254]:
prediction_y=rf.predict(normval_x)
r2 = calculate_oos(val_y,prediction_y)

In [255]:
r2

-0.2183133882912116